# Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os
path= '/content/drive/MyDrive/codigos/Anal-tica-en-Recursos-Humanos'
os.chdir(path)
sys.path.append(path+'/')

# Visual Studio

In [16]:
import os
import sys

ruta =os.getcwd()
sys.path.append(ruta)

# Despliegue

In [ ]:
import a_funciones as fn ### importar modulo de  funciones
from a_funciones import * ### importar toddas llas funciones del archivo
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
!pip install openpyxl
import openpyxl

import numpy as np

El despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
En este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.

In [2]:
#### conectarse a base de datos preprocesada

con=sql.connect('bases de datos RRHH/datos.db')
cur=con.cursor()

In [3]:
#crear dataframe para el modelado(2015)
df = pd.read_sql("""select * from completo
where InfoDate='2016-12-31'
""",con=con)

In [4]:
# Hacer la preparacion de los datos nuevos
df_t= fn.preparar_datos(df)

In [5]:
# Cargar modelo entrenado
rf_final = joblib.load("salidas\\rf_final.pkl")

In [10]:
# Realizar predicciones
predicciones = rf_final.predict(df_t)
pd_pred = pd.DataFrame(predicciones, columns=['Atrition'])

# Crear DataFrame con predicciones
perf_pred = pd.concat([df['EmployeeID'], df_t, pd_pred], axis=1)

# Guardar predicciones en archivos
perf_pred[['EmployeeID', 'Atrition']].to_excel("salidas\\prediccion.xlsx")
#Guardar importancia de las caracteristicas a la hora de predecir
feature_names = df_t.columns


In [11]:
importances = pd.DataFrame({'Feature': feature_names, 'Importance': rf_final.feature_importances_})
importances.to_excel("salidas\\importances.xlsx")

In [12]:
# Ver las 10 predicciones más bajas
emp_pred_bajo = perf_pred.sort_values(by=["Atrition"], ascending=True).head(10)
emp_pred_bajo.set_index('EmployeeID', inplace=True)
pred = emp_pred_bajo.T
print(pred)

EmployeeID                      1      2800      2801      2802      2803  \
Age                      1.541369 -0.101159 -0.101159  0.665354 -0.758170   
DistanceFromHome        -0.393938 -0.270544 -1.010909 -0.764121  0.099639   
EnvironmentSatisfaction  0.252200 -0.665460 -1.583120  0.252200 -1.583120   
JobSatisfaction          1.156302  1.156302  1.156302  1.156302  0.246202   
MonthlyIncome            1.405136 -0.810167  2.350878 -0.870085 -0.783819   
NumCompaniesWorked      -0.677862 -1.078797  0.925878 -0.677862 -0.677862   
PercentSalaryHike       -1.150554 -1.150554 -0.603911 -0.877232  2.402628   
TotalWorkingYears       -1.320847 -0.163984 -0.549605  1.764122 -0.163984   
YearsSinceLastPromotion -0.679146 -0.058285 -0.679146  0.562576 -0.679146   
YearsWithCurrManager    -1.155935  1.086895 -1.155935  2.208310  1.367249   
Atrition                 0.000000  0.000000  0.000000  0.000000  0.000000   

EmployeeID                   2804      2806      2808      2811      2812  